## Optimising Dream 11 Team Selection ##

### Section 1.1 Data Preparation :
1) Prepare a Score Vector (The mean of a Player's Score across 10 Matches is considered) <br>
2) Prepare Standard Deviation Vector( to capture variability or inconsistencies in player's scores). This is to be used in Mean Variance Optimisation

In [3]:
pip install gurobipy

   ---------------------------------------- 0.0/10.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.3 MB 1.3 MB/s eta 0:00:09
    --------------------------------------- 0.2/10.3 MB 2.5 MB/s eta 0:00:05
   -- ------------------------------------- 0.7/10.3 MB 5.2 MB/s eta 0:00:02
   --- ------------------------------------ 1.0/10.3 MB 5.7 MB/s eta 0:00:02
   ----- ---------------------------------- 1.4/10.3 MB 6.1 MB/s eta 0:00:02
   ----- ---------------------------------- 1.5/10.3 MB 6.4 MB/s eta 0:00:02
   -------- ------------------------------- 2.2/10.3 MB 7.3 MB/s eta 0:00:02
   --------- ------------------------------ 2.5/10.3 MB 7.3 MB/s eta 0:00:02
   ---------- ----------------------------- 2.7/10.3 MB 7.1 MB/s eta 0:00:02
   ----------- ---------------------------- 2.9/10.3 MB 6.9 MB/s eta 0:00:02
   ------------ --------------------------- 3.3/10.3 MB 7.0 MB/s eta 0:00:01
   -------------- ------------------------- 3.7/10.3 MB 7.0 MB/s eta 0:00:01
   ---

In [1]:
import pandas as pd
import numpy as np
from gurobipy import *
#str = unicode(str, errors='ignore')
data_set_df=pd.read_csv(r"D:\DSML\5. Optimisation for ML\Players_Score_Names-1.csv", encoding='gbk')
data_set_df.head()

,Player Name,Player Type,Team,Retrospective Scores_10,Retrospective Scores_1,Retrospective Scores_2,Retrospective Scores_3,Retrospective Scores_4,Retrospective Scores_5,Retrospective Scores_6,Retrospective Scores_7,Retrospective Scores_8,Retrospective Scores_9,Price
0,Virat Kohli,BAT,INDIA,145,128,41,55,85,56,3,122,4,54,17
1,Rohit Sharma,BAT,INDIA,46,48,86,131,0,81,6,53,56,74,15
2,Shubman Gill,BAT,INDIA,26,53,16,104,74,27,121,19,58,67,8
3,Shreyas Iyer,BAT,INDIA,33,19,53,25,0,48,105,3,14,38,12
4,Surya Kumar Yadav,BAT,INDIA,2,8,72,50,26,35,24,19,0,0,12


#### Find mean and Standard Deviation of the scores of all 30 Players ###

In [3]:
score_df=data_set_df.iloc[0:,3:data_set_df.shape[1]-1] # Separate the score data from other details
score_df

,Retrospective Scores_10,Retrospective Scores_1,Retrospective Scores_2,Retrospective Scores_3,Retrospective Scores_4,Retrospective Scores_5,Retrospective Scores_6,Retrospective Scores_7,Retrospective Scores_8,Retrospective Scores_9
0,145,128,41,55,85,56,3,122,4,54
1,46,48,86,131,0,81,6,53,56,74
2,26,53,16,104,74,27,121,19,58,67
3,33,19,53,25,0,48,105,3,14,38
4,2,8,72,50,26,35,24,19,0,0
5,77,84,19,172,26,52,83,19,39,111
6,47,25,31,18,23,5,33,87,77,55
7,11,55,137,70,32,80,65,26,50,8
8,36,3,41,1,25,75,25,25,25,100
9,39,50,50,16,75,8,75,25,25,50


In [5]:
sd_score=[np.std(score_df.iloc[i,:].values) for i in range(score_df.shape[0])] #Standard Deviation to measure inconsistency
scores=[np.mean(score_df.iloc[i,:].values) for i in range(score_df.shape[0])] #Score of each player
print(scores)

[69.3, 58.1, 56.5, 33.8, 23.6, 68.2, 40.1, 53.4, 35.6, 41.3, 47.5, 58.8, 40.3, 45.6, 30.2, 23.7, 50.8, 34.3, 25.1, 33.9, 36.8, 29.7, 22.4, 40.4, 35.5, 33.0, 36.2, 45.0, 13.5, 28.5]


In [7]:
#########Other Parameters Set-up############

# Intialise a Player Type(Batsman,Bowler,Allrounder or Wicket Keeper) Matrix
ptype=np.zeros((4,30))

# Initialize a Team Matrix (The Dream 11 Team has to be formed from two competing Teams)
team=np.zeros((2,30))

# Fill in the Player and Team matrices with values from Dataset
for i in range(data_set_df.shape[0]):
    if(data_set_df.loc[i,'Player Type']=='BAT'):
        ptype[0,i]=1
    elif(data_set_df.loc[i,'Player Type']=='BOWL'):
        ptype[1,i]=1
    elif(data_set_df.loc[i,'Player Type']=='AR'):
        ptype[2,i]=1
    else:
        ptype[3,i]=1
    if(data_set_df.loc[i,'Team']=='INDIA'):
        team[0,i]=1
    else:
        team[1,i]=1

# Prepare a Budget Vector
price=data_set_df['Price'].values
print(price)

# Prepare the lower bound and upper bound for Player Type . This is to be read as :
# The no. of batsman can be between 3 and 6 (lb[0] and ub[0] respectively)
# Similarly, the no. of bowlers can be between 3 and 6; No. of allrounders can be between 1 and 4; 
# and No of wicketkeeper can be between 1 and 4
lb=[3,3,1,1]
ub=[6,6,4,4]

# There can only be 1 Captain, 1 Vice Captain and the rest 9 players will be considered as normal players
player_role=[9,1,1]


N=data_set_df.shape[0] # No of Players available 
M=len(player_role) #No of Player Roles (N,VC,C)
K=ptype.shape[0] #No. of Player Types (4-BAT,BOWL,AR,WK)

# Risk_aversion captures the risk tolerance of a Dream 11 Player. 
#From a problem perspective, we aim to punish a player for his inconsistency. Risk_aversion captures the degree of punishment

risk_aversion=0

[17 15  8 12 12 17 17 15 10 16 12  5  8  2  4  6  2  2 11  8 16  8  8  2
 18  4  1  1  1  7]


### Section 1.2 Stating the Objective Function and Constraints. Displaying the Output ###

In [10]:
def dream11_optimisation():
    data_set = Model()

    # Creat variables
    x=data_set.addVars(M, N, vtype=GRB.BINARY, name = "x")


    # Set objective : A Vice Captain has his score multiplied by 1.5 and A captain has his score multiplied by 2.
    # Additionally, a Player is punished for his inconsistency (A form of Mean Variance Optimisation, 
    # used mostly in financial portfolio management)
    data_set.setObjective( quicksum(scores[j]*x[0,j] for j in range(N))\
                  +quicksum(1.5*scores[j]*x[1,j] for j in range(N))\
                  +quicksum(2*scores[j]*x[2,j] for j in range(N))\
                  -risk_aversion*(quicksum(x[i,j]*sd_score[j] for i in range(M) for j in range(N))), GRB.MAXIMIZE)
    
    # The Player can either not be selected and if selected, can only take in one of the role : Either a Captain, Vice Captain or
    # an ordinary player
    data_set.addConstrs(( quicksum(x[i,j] for i in range(M)) <= 1 for j in range(N) )) ##equation 3
    
    # The below constraint ensures that there are 9 ordinary players, only 1 Vice Captain and only 1 Captain in the Team
    data_set.addConstrs(quicksum(x[i,j] for j in range(N)) ==player_role[i] for i in range(M))
    
    # Only 100 Points are available for Spending
    data_set.addConstr(quicksum(price[j]*x[i,j] for i in range(M) for j in range(N)) <=100) ##equation2
    
    # The no of Batsman,Bowler, Allrounder and WK has to be within the specified ranges
    data_set.addConstrs(quicksum(x[i,j]*ptype[k,j] for i in range(M) for j in range(N))<=ub[k] for k in range(K)) ##equation 6

    data_set.addConstrs(quicksum(x[i,j]*ptype[k,j] for i in range(M) for j in range(N))>=lb[k] for k in range(K)) ##equation 6

    # There can be a maximum of 7 players from one team
    data_set.addConstrs(quicksum(x[i,j]*team[k,j] for i in range(M) for j in range(N))<=7 for k in range(team.shape[0])) ##equation 5

    return data_set,x


In [12]:
# risk_aversion can vary between 1 to 10: 
# A risk_aversion score of 0 means the risk taking apetite is huge. This Dream 11 Player doesn't take inconsistencies in 
# data_setket players performance into account while selecting his Playing 11. 
# 10 signifies that the risk-taking apetite of the Dream 11 player is almost negligible. He/She would prefer only consistent
# players in his/her playing 11
while (True):
    risk_aversion = int(input("Enter your risk tolerance : "))
    if(risk_aversion>10 or risk_aversion < 0 ):
        print("Invalid Input!!. Please Choose a tolerance limit between 0 and 10"+"(both inclusive)")
        continue
    else:
        print ('Risk Tolerance of the Dream-11 Player is',risk_aversion)
        # setup the model again
        dr_data_set,x = dream11_optimisation()
        

        
        # Solving the model
        dr_data_set.optimize()
        
        # Display the Output
        
    
        print('The Optimum Team for the upcoming match based on your risk tolerance is shown below:- \n')
        for v in dr_data_set.getVars():
            print(v)
            if(int(v.VarName[2])==2 and v.x > 0):
                index=int(v.VarName[v.VarName.index(',')+1:v.VarName.index(']')])
                print('The Captain of the Team is',data_set_df.loc[index,'Player Name'] + \
                  '('+ data_set_df.loc[index,'Player Type'] +')'+ ' from '+ data_set_df.loc[index,'Team'])
        for v in dr_data_set.getVars():
            if(int(v.VarName[2])==1 and v.x > 0):
                index=int(v.VarName[v.VarName.index(',')+1:v.VarName.index(']')])
                print('The Vice Captain of the Team is',data_set_df.loc[index,'Player Name'] + \
                      '('+ data_set_df.loc[index,'Player Type'] +')'+ ' from '+ data_set_df.loc[index,'Team']+'\n')
        print('Other Players:- \n')
        for v in dr_data_set.getVars():
            if(int(v.VarName[2])==0 and v.x > 0):
                index=int(v.VarName[v.VarName.index(',')+1:v.VarName.index(']')])
                print(data_set_df.loc[index,'Player Name'] +'('+ data_set_df.loc[index,'Player Type'] +')'+ \
                      ' from '+ data_set_df.loc[index,'Team'])
        
        # Ask from User if he want's to select a team on other risk tolerance levels, apart from the one entered earlier
        choice = input("Enter (Y/N) if you wan't to see team combination for other risk_tolerance levels : \n")
        if(choice.upper()=='Y' ):
            continue
        else:
            print('No more Team Selections')
            print(dr_data_set)
            break

Enter your risk tolerance :  1


Risk Tolerance of the Dream-11 Player is 1
Restricted license - for non-production use only - expires 2025-11-24
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i5-1135G7 @ 2.40GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 44 rows, 90 columns and 540 nonzeros
Model fingerprint: 0x5c8bd5c9
Variable types: 0 continuous, 90 integer (90 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [9e-02, 9e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective 114.1416021
Presolve time: 0.02s
Presolved: 44 rows, 90 columns, 531 nonzeros
Variable types: 0 continuous, 90 integer (90 binary)
Found heuristic solution: objective 255.1054355

Root relaxation: objective 2.959078e+02, 57 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Curre

Enter (Y/N) if you wan't to see team combination for other risk_tolerance levels : 
 N


No more Team Selections
<gurobi.Model MIP instance Unnamed: 44 constrs, 90 vars, No parameter changes>
